In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA-2-RegExp

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Tuple } from "recursive-set";
import {
  State,
  Char,
  DFA,
  RegExp,
  TransRelDet,
  key,
  dfa2regexp,
  BinaryOp,
  UnaryOp
} from "./05-DFA-2-RegExp";

import { dfa2dot } from "./FSM-2-Dot";

In [ ]:
// Singletons {1}, {2}, {3}
const S0 = new RecursiveSet(1);
const S1 = new RecursiveSet(2);
const S2 = new RecursiveSet(3);

// Q = { {1}, {2}, {3} }
const Q = new RecursiveSet<RecursiveSet<number>>(S0, S1, S2);

// Σ = {a, b}
const Sigma = new RecursiveSet<string>("a", "b");

// δ (Übergangsfunktion)
const delta: TransRelDet = new Map<string, RecursiveSet<number>>();
delta.set(key(S0, "a"), S1); // (1,a)->2
delta.set(key(S1, "b"), S2); // (2,b)->3
delta.set(key(S2, "a"), S1); // (3,a)->2

// Startzustand und akzeptierende Zustände
const q0 = S0;
const A = new RecursiveSet<RecursiveSet<number>>(S2);

// DFA Objekt
const dfa: DFA = { Q, Sigma, delta, q0, A };

In [ ]:
const { dot } = dfa2dot(dfa);
const viz = await instance();
display.html(viz.renderString(dot, { format: "svg" }));

In [ ]:
const r: RegExp = dfa2regexp(dfa);

console.log("Unvereinfachter regulärer Ausdruck:");
r;

## Helpfunctions:

In [ ]:
function isAtom(r: RegExp): r is number | string {
    return typeof r === 'number' || typeof r === 'string';
}

function isKleene(r: RegExp): r is Tuple<[RegExp, UnaryOp]> {
    return r instanceof Tuple && r.length === 2 && r.values[1] === '*';
}

function isBinary(r: RegExp): r is Tuple<[RegExp, BinaryOp, RegExp]> {
    return r instanceof Tuple && r.length === 3;
}

function isZero(r: RegExp): boolean {
    return r === 0;
}

function isEps(r: RegExp): boolean {
    return r === "ε" || r === "𝜀";
}

function eq(a: RegExp, b: RegExp): boolean {
    return JSON.stringify(a) === JSON.stringify(b);
}

As this regular expression is nearly unreadable, the function `simplify` can be used to simplify this expression.

In [ ]:
function simplify(r: RegExp): RegExp {
  if (isAtom(r)) return r;

  if (isKleene(r)) {
    const innerRaw = r.values[0];
    let inner = simplify(innerRaw);

    if (isZero(inner)) return "ε"; 
    if (isEps(inner)) return "ε"; 
    if (isKleene(inner)) return inner;

    if (isBinary(inner) && inner.values[1] === '+') {
        const left = inner.values[0];
        const right = inner.values[2];
        
        if (isEps(left) || isZero(left)) {
            return new Tuple<[RegExp, UnaryOp]>(simplify(right), '*');
        }
        if (isEps(right) || isZero(right)) {
            return new Tuple<[RegExp, UnaryOp]>(simplify(left), '*');
        }
    }

    return new Tuple<[RegExp, UnaryOp]>(inner, '*');
  }

  if (isBinary(r)) {
    const left = simplify(r.values[0]);
    const op = r.values[1];
    const right = simplify(r.values[2]);

    if (op === "+") {
      if (isZero(left)) return right;
      if (isZero(right)) return left;
      if (eq(left, right)) return left;
      return new Tuple<[RegExp, BinaryOp, RegExp]>(left, '+', right);
    }

    if (op === "⋅") {
      if (isZero(left) || isZero(right)) return 0; 
      if (isEps(left)) return right;
      if (isEps(right)) return left;
      return new Tuple<[RegExp, BinaryOp, RegExp]>(left, '⋅', right);
    }
  }

  return r;
}

In [ ]:
let s = simplify(r);
s = simplify(s); // Zweiter Durchlauf für verschachtelte Vereinfachungen

console.log("Vereinfachter Ausdruck (Struktur):");
s;

The function `regexp_2_string` takes a regular expression that is represented as a nested tuple and transforms it into a string.

In [ ]:
function regexpToString(r: RegExp): string {
  if (r === 0) return "0";
  if (r === "ε" || r === "𝜀") return "𝜀";
  if (isAtom(r)) return r.toString();

  if (isKleene(r)) {
    const inner = r.values[0];
    const sInner = regexpToString(inner);

    if (typeof inner === "string") {
        return sInner + "*";
    } else {
        return "(" + sInner + ")*";
    }
  }

  if (isBinary(r)) {
    const left = r.values[0];
    const op = r.values[1];
    const right = r.values[2];

    const s1 = regexpToString(left);
    const s2 = regexpToString(right);

    if (op === "⋅") {
      return s1 + "⋅" + s2;
    }

    if (op === "+") {
      return "(" + s1 + "+" + s2 + ")";
    }
  }

  throw new Error(`${JSON.stringify(r)} is not a suitable regular expression`);
}

In [ ]:
console.log("\nEndgültiger regulärer Ausdruck (String):");
regexpToString(s);